In [1]:
library(foreign)
library(car)
library(lmtest)
library(AER)
library(caret)
# library(caTools)
library(reshape2)
library(stargazer)
library(mlogit)
library(nnet)

Warning message:
"package 'car' was built under R version 4.2.3"
Loading required package: carData

Warning message:
"package 'carData' was built under R version 4.2.2"
Warning message:
"package 'lmtest' was built under R version 4.2.2"
Loading required package: zoo

Warning message:
"package 'zoo' was built under R version 4.2.3"

Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Warning message:
"package 'AER' was built under R version 4.2.2"
Loading required package: sandwich

Warning message:
"package 'sandwich' was built under R version 4.2.2"
Loading required package: survival

Warning message:
"package 'caret' was built under R version 4.2.3"
Loading required package: ggplot2

Warning message:
"package 'ggplot2' was built under R version 4.2.3"
Loading required package: lattice


Attaching package: 'caret'


The following object is masked from 'package:survival':

    cluster


Warning message:
"package 'reshape2' wa

## MÔ HÌNH MULTINOMIAL LOGITIC  Data: hsb2.csv
# These are the Madsen data used in the revised lecture notes. High School and Beyond survey
Description
Two hundred observations were randomly sampled from the High School and Beyond survey, a survey conducted on high school seniors by the National Center of Education Statistics. Các biến số: 200 observations and 10 variables: gender:levels female and male; race:levels african american, asian, hispanic, and white; ses:Socio economic status of student's family, with levels low, middle, and high; schtyp: Type of school, with levels public and private; prog: Type of program, with levels general, academic, and vocational; read: Standardized reading score; write:Standardized writing score; math:Standardized math score; science:Standardized science score; socst:Standardized social studies score.
## INPUT DATA



In [2]:
hsb2=read.csv("hsb2.csv", header=TRUE)
head(hsb2,2)
dim(hsb2)

,gender,race,ses,schtyp,prog,read,write,math,science,socst
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>
1,male,white,low,public,general,57,52,41,47,57
2,female,white,middle,public,vocational,68,59,53,63,61


[1] 200  10

In [3]:
# table
print('Checking the output (dependent) variable')
table(hsb2$prog) 
print('Bảng phân bổ prog và loại trường schtyp')
with(hsb2, table(prog, schtyp))
print('Kiểm định sự tương quan giưa các biến số') 
chisq.test(hsb2$prog,hsb2$ses)
chisq.test(hsb2$prog,hsb2$gender)
chisq.test(hsb2$prog,hsb2$schtyp)

[1] "Checking the output (dependent) variable"



  academic    general vocational 
       105         45         50 

[1] "Bảng phân bổ prog và loại trường schtyp"


            schtyp
prog         private public
  academic        24     81
  general          6     39
  vocational       2     48

[1] "Kiểm định sự tương quan giưa các biến số"



	Pearson's Chi-squared test

data:  hsb2$prog and hsb2$ses
X-squared = 16.604, df = 4, p-value = 0.002307



	Pearson's Chi-squared test

data:  hsb2$prog and hsb2$gender
X-squared = 0.052809, df = 2, p-value = 0.9739



	Pearson's Chi-squared test

data:  hsb2$prog and hsb2$schtyp
X-squared = 9.2687, df = 2, p-value = 0.009712


## UOC LUONG MULTINOMIAL LOGIT 

In [4]:
multihsb2=multinom(prog ~ ses + write, data=hsb2)

# weights:  15 (8 variable)
initial  value 219.722458 
iter  10 value 179.983731
final  value 179.981726 
converged


## MỘT SỐ PHÂN TÍCH 

In [5]:

print('Trình bày dạng cột') 

stargazer(multihsb2, type="text")

print('Tính tỷ số OR')

multihsb2.OR = exp(coef(multihsb2))
multihsb2.OR

print('Tỷ số OR cho từng nhóm')
stargazer(multihsb2, coef=list(multihsb2.OR), type="text", p.auto=FALSE)

[1] "Trình bày dạng cột"

                      Dependent variable:     
                  ----------------------------
                     general      vocational  
                       (1)            (2)     
----------------------------------------------
seslow               1.163**        0.983*    
                     (0.514)        (0.596)   
                                              
sesmiddle             0.630         1.274**   
                     (0.465)        (0.511)   
                                              
write               -0.058***      -0.114***  
                     (0.021)        (0.022)   
                                              
Constant              1.689        4.236***   
                     (1.227)        (1.205)   
                                              
----------------------------------------------
Akaike Inf. Crit.    375.963        375.963   
Note:              *p<0.1; **p<0.05; ***p<0.01
[1] "Tính tỷ số OR"


,(Intercept),seslow,sesmiddle,write
general,5.416653,3.199009,1.876792,0.9437152
vocational,69.101326,2.671709,3.575477,0.8926115


[1] "Tỷ số OR cho từng nhóm"

                      Dependent variable:     
                  ----------------------------
                     general      vocational  
                       (1)            (2)     
----------------------------------------------
seslow               3.199**        2.672*    
                     (0.514)        (0.596)   
                                              
sesmiddle             1.877         3.575**   
                     (0.465)        (0.511)   
                                              
write                0.944***      0.893***   
                     (0.021)        (0.022)   
                                              
Constant              5.417        69.101***  
                     (1.227)        (1.205)   
                                              
----------------------------------------------
Akaike Inf. Crit.    375.963        375.963   
Note:              *p<0.1; **p<0.05; ***p<0.01


In [6]:

## XÁC SUẤT TRUNG BÌNH CÁC LỰA CHỌN CHƯƠNG TRÌNH
Prob=fitted(multihsb2)
head(Prob,3)
print('trung bình xác suất lựa chọn=academic')
P_aca_TB=mean(Prob[,c(1)])
P_aca_TB
print('trung bình xác suất lựa chọn=general')
P_gen_TB=mean(Prob[,c(2)])
P_gen_TB
print('trung bình xác suất lựa chọn=vocation=')
P_voc_TB=mean(Prob[,c(3)])
P_voc_TB

,academic,general,vocational
1,0.4247920,0.3619528,0.2132552
2,0.6110350,0.2036240,0.1853410
3,0.2917517,0.2336122,0.4746361


[1] "trung bình xác suất lựa chọn=academic"


[1] 0.5249988

[1] "trung bình xác suất lựa chọn=general"


[1] 0.2250004

[1] "trung bình xác suất lựa chọn=vocation="


[1] 0.2500008

In [7]:
dwrite <- data.frame(ses = rep(c("low", "middle", "high"), each = 21), write = rep(c(30:50),3))
# dwrite
pp.write <- cbind(dwrite, predict(multihsb2, newdata = dwrite, type = "probs", se = TRUE))
by(pp.write[, 3:5], pp.write$ses, colMeans)

pp.write$ses: high
  academic    general vocational 
 0.4418634  0.2256730  0.3324636 
------------------------------------------------------------ 
pp.write$ses: low
  academic    general vocational 
 0.2247991  0.3541661  0.4210348 
------------------------------------------------------------ 
pp.write$ses: middle
  academic    general vocational 
 0.2302619  0.2096853  0.5600528 